<a href="https://colab.research.google.com/github/Joshuajee/AI-ML-PROJECTS/blob/master/GET_DATA_From_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gpt4all
!pip install "gpt4all[cuda]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.1 MB/s eta 0:00:00


In [ ]:
import requests
import json
from google.colab import drive
from gpt4all import GPT4All
import torch

has_gpu = torch.cuda.is_available()

has_gpu  # Should return True

In [ ]:
# Get Prompt
reponse = requests.get("https://raw.githubusercontent.com/Joshuajee/AI-ML-PROJECTS/master/data/llms/prompt.csv")
if reponse.status_code == 200:
  with open("prompt.csv", "wb") as file:
    file.write(reponse.content)
else:
    raise Exception("Error: downloading", reponse.status_code)

In [ ]:
# Uncomment to use google drive,
#drive.mount('/content/drive/') # Mount your Google Drive to the local /content/drive directory
#drive_path = "/content/drive/My Drive/Colab Notebooks/assessment-2" # Change to a google drive path on your system

In [ ]:
# To run the code locally use this instead
drive_path = "/"

In [ ]:
def create_llm_response(model, batch_size = 4):
    '''
    This function go through the prompt CSV file and
    save the response of the LLM inside a CSV file.
    it always start from where it stops so canceling the process will not lead to lose
    '''
    if has_gpu:
        gpt_model = GPT4All(model, device="cuda") # downloads / loads a 4.66GB LLM and run on GPU
    else:
        gpt_model = GPT4All(model) # downloads / loads a 4.66GB LLM

    try:
      model_file = open(drive_path + model + ".csv", "a+")
      model_file.seek(0)
    except:
      model_file = open(drive_path + model + ".csv", "x+")

    last_line = len(model_file.readlines())
    model_file.seek(last_line)
    current_line = last_line
    with open("prompt.csv", "r") as file:
        lines = file.readlines()
        while current_line <= 1000:
            print(f"Job progress: {current_line/10} %")
            line_split = lines[current_line].split(",")
            prompt = "Write an article on " + line_split[2].replace("\n", "")
            if current_line == 0:
                model_file.write(line_split[0] + "," + line_split[1] + "," + line_split[2].replace('\n', '') + ", Prompt, Response, Model\n")
            else:
                with gpt_model.chat_session():
                    answer = gpt_model.generate(prompt, max_tokens=1024)
                    answer = answer.replace("\n", "").replace('"', "'")
                    model_file.write(line_split[0] + "," + line_split[1] + ',"' + str((line_split[2]).strip()) +  '","' + str(prompt.strip()) + '","' + str(answer.strip()) + '",' + model + " \n")
            current_line += 1
            #To save after every 10 lines that is processed
            if current_line % batch_size == 0:
                model_file.close()
                model_file = open(drive_path + model + ".csv", "a+")
                print(f"Batch {(current_line // batch_size) + 1} saved")
        print(f"Job completed")
    model_file.close()

In [ ]:
create_llm_response("Meta-Llama-3-8B-Instruct.Q4_0.gguf")

In [ ]:
create_llm_response("Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf")

In [ ]:
create_llm_response("Phi-3-mini-4k-instruct.Q4_0.gguf")

In [ ]:
create_llm_response("orca-mini-3b-gguf2-q4_0.gguf")